In [ ]:

import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import csv# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler#Keras
import math
import keras
from keras import models
from keras import layers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"amarshukla7","key":"3e21fa8cec03b066cff091cc91e46b2d"}'}

In [ ]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download andradaolteanu/gtzan-dataset-music-genre-classification -p music_dataset


100% 1.21G/1.21G [00:09<00:00, 116MB/s]
100% 1.21G/1.21G [00:09<00:00, 134MB/s]


In [ ]:
!unzip /content/music_dataset/gtzan-dataset-music-genre-classification.zip -d /content/music_dataset/gtzan

Archive:  /content/music_dataset/gtzan-dataset-music-genre-classification.zip
  inflating: /content/music_dataset/gtzan/Data/features_30_sec.csv  
  inflating: /content/music_dataset/gtzan/Data/features_3_sec.csv  
  inflating: /content/music_dataset/gtzan/Data/genres_original/blues/blues.00000.wav  
  inflating: /content/music_dataset/gtzan/Data/genres_original/blues/blues.00001.wav  
  inflating: /content/music_dataset/gtzan/Data/genres_original/blues/blues.00002.wav  
  inflating: /content/music_dataset/gtzan/Data/genres_original/blues/blues.00003.wav  
  inflating: /content/music_dataset/gtzan/Data/genres_original/blues/blues.00004.wav  
  inflating: /content/music_dataset/gtzan/Data/genres_original/blues/blues.00005.wav  
  inflating: /content/music_dataset/gtzan/Data/genres_original/blues/blues.00006.wav  
  inflating: /content/music_dataset/gtzan/Data/genres_original/blues/blues.00007.wav  
  inflating: /content/music_dataset/gtzan/Data/genres_original/blues/blues.00008.wav  
  

In [ ]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [ ]:
df=pd.read_csv('/content/data.csv')
df.head()


,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label


In [ ]:
!gsutil cp -r  /content/music_dataset/gtzan /content/drive/MyDrive

Copying file:///content/music_dataset/gtzan/Data/features_3_sec.csv...
Copying file:///content/music_dataset/gtzan/Data/features_30_sec.csv...
Copying file:///content/music_dataset/gtzan/Data/images_original/metal/metal00004.png...
Copying file:///content/music_dataset/gtzan/Data/images_original/metal/metal00051.png...
/ [4 files][ 11.8 MiB/ 11.8 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file:///content/music_dataset/gtzan/Data/images_original/metal/metal00043.png...
Copying file:///content/music_dataset/gtzan/Data/images_original/metal/metal00030.png...
Copying file:///content/music_dataset/gtzan/Data/images_original/metal/metal00093.png...
Copying file:///content/music_dataset/gtzan/Data/images_original/metal/metal000

In [ ]:

genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    
    for filename in os.listdir(f'/content/drive/MyDrive/gtzan/Data/genres_original/{g}'):
        songname = f'/content/drive/MyDrive/gtzan/Data/genres_original/{g}/{filename}'
        
        print(filename)
        if(filename=='.ipynb_checkpoints'):
          continue
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        rmse = librosa.feature.rms(y=y)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

blues.00052.wav
blues.00099.wav
blues.00014.wav
blues.00037.wav
blues.00012.wav
blues.00055.wav
blues.00039.wav
blues.00026.wav
blues.00004.wav
blues.00020.wav
blues.00064.wav
blues.00030.wav
blues.00028.wav
blues.00048.wav
blues.00045.wav
blues.00049.wav
blues.00094.wav
blues.00073.wav
blues.00085.wav
blues.00097.wav
blues.00067.wav
blues.00005.wav
blues.00083.wav
blues.00082.wav
blues.00074.wav
blues.00060.wav
blues.00068.wav
blues.00034.wav
blues.00031.wav
blues.00035.wav
blues.00050.wav
blues.00006.wav
blues.00001.wav
blues.00032.wav
blues.00011.wav
blues.00033.wav
blues.00086.wav
blues.00010.wav
blues.00043.wav
blues.00025.wav
blues.00053.wav
blues.00095.wav
blues.00098.wav
blues.00078.wav
blues.00066.wav
blues.00063.wav
blues.00056.wav
blues.00009.wav
blues.00042.wav
blues.00079.wav
blues.00041.wav
blues.00000.wav
blues.00058.wav
blues.00024.wav
blues.00007.wav
blues.00046.wav
blues.00038.wav
blues.00059.wav
blues.00008.wav
blues.00054.wav
blues.00084.wav
blues.00047.wav
blues.00

In [ ]:
data1 = pd.read_csv('/content/drive/MyDrive/data.csv')
data1 = data1.drop_duplicates(keep='last')
data1[50:100]

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
1154,blues.00041.wav,0.386950,0.129172,2390.772194,2463.394088,5403.783991,0.102289,-107.161522,88.466171,-4.174954,55.484200,-8.007363,38.415665,-6.399442,28.574484,-19.820578,13.247096,-19.241852,12.072803,-4.580049,3.791739,-16.991062,11.624590,-7.360203,-3.657303,-7.177438,1.977192,blues
1155,blues.00000.wav,0.349943,0.130225,1784.420446,2002.650192,3806.485316,0.083066,-113.596748,121.557297,-19.158825,42.351032,-6.376458,18.618876,-13.697912,15.344631,-12.285267,10.980492,-8.324325,8.810669,-3.667368,5.751691,-5.162763,0.750948,-1.691938,-0.409953,-2.300209,1.219929,blues
1156,blues.00058.wav,0.356588,0.162028,2068.371125,2033.930047,4231.499413,0.104372,-95.436028,105.216698,-26.953011,60.830875,-4.828223,21.538469,-13.514777,12.122107,-20.233633,12.138657,-5.227018,10.262574,-9.438187,3.867534,-10.187452,3.588093,-3.100752,0.194597,0.842872,-3.416576,blues
1157,blues.00024.wav,0.263129,0.086982,1120.387619,1447.839934,2325.102607,0.047298,-274.288757,130.417419,-6.394800,70.410683,13.265855,9.463105,7.723187,-1.245347,-0.575020,-2.299992,3.783482,0.080915,-0.939682,2.295373,4.290513,-1.661925,-10.444824,-6.797826,-1.068164,-1.596130,blues
1158,blues.00007.wav,0.307921,0.131785,1451.754147,1577.369917,2955.348796,0.061435,-179.395432,136.459244,-26.656363,39.988029,5.289677,10.924428,-20.561890,8.513765,-11.356910,-3.469077,-8.414556,-6.954826,-3.544536,-8.051242,-8.959538,-8.424336,-10.558884,-10.788157,-4.693748,-8.638613,blues
1159,blues.00046.wav,0.387851,0.102582,1912.506881,2361.474653,4776.862782,0.058426,-182.638275,90.676796,14.119084,67.549828,2.693214,38.430527,-8.410898,24.127445,-13.781991,4.835997,-16.682030,5.811708,-5.149533,0.192984,-14.480762,6.862884,-4.442362,-2.986889,-4.816248,5.400325,blues
1160,blues.00038.wav,0.265883,0.168145,1513.422107,2140.606779,3449.679140,0.044378,-192.667175,111.190765,21.372723,26.398340,-0.646122,23.486736,-25.464550,16.937731,-16.977055,5.788635,-25.730206,-5.044726,-17.631821,-7.921688,-15.152404,-12.342546,-17.227766,-7.771727,-13.033959,-17.945749,blues
1161,blues.00059.wav,0.405961,0.169987,2235.906951,2100.004335,4390.031787,0.142288,-86.225304,91.948448,-23.408659,68.253288,-3.895873,22.745262,-9.899613,16.895977,-11.397531,18.431677,-7.015263,10.771054,-11.938685,1.005814,-5.923496,5.318165,-6.140526,1.948991,-5.102755,-0.666248,blues
1162,blues.00008.wav,0.409037,0.142438,1719.213163,2031.643884,3781.318802,0.064028,-121.361023,122.513107,-14.742128,46.143444,-8.165333,20.176525,-19.172476,23.055616,-11.830503,21.177010,-6.721190,7.010945,-12.741831,5.066004,-5.175478,-1.444024,-7.037070,2.697451,3.409809,-2.698353,blues
1163,blues.00054.wav,0.452728,0.200889,2030.485224,1987.320416,4062.942099,0.103386,-57.978794,108.989113,-36.200199,64.032860,1.454682,26.300751,-17.470224,20.423563,-9.526501,18.171871,-6.047029,13.862625,-11.287124,4.663286,-6.388999,3.213776,-8.078659,-1.723661,-5.422675,-2.582366,blues


In [ ]:
!gsutil cp   /content/data.csv /content/drive/MyDrive


Copying file:///content/data.csv...
/ [1 files][  1.0 MiB/  1.0 MiB]                                                
Operation completed over 1 objects/1.0 MiB.                                      


(995, 28)

In [ ]:
data1.shape

(995, 28)

In [ ]:
data1 = data1.drop(['filename'],axis=1)
data1.head()

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
1104,0.339075,0.098910,1443.265285,1670.369919,3069.356440,0.067194,-225.269730,117.645897,-14.543757,64.804359,12.927466,19.029963,10.684547,9.884662,-2.385688,11.688362,1.542959,3.050155,-7.630156,-8.957814,-4.352594,-0.062078,-2.554219,-3.437579,-3.866575,1.316515,blues
1105,0.370279,0.105061,1538.817338,1703.507382,3175.272466,0.074799,-177.165863,136.221588,-26.915182,36.376289,-18.760828,20.909632,-1.778092,3.013012,-5.600873,5.638467,-8.117091,5.642528,-10.790227,-2.416264,-6.943611,-2.507238,-5.203642,-3.407661,-3.246502,-2.681821,blues
1106,0.329163,0.066981,1172.065695,1705.817502,2344.952487,0.045046,-305.705170,113.060928,12.118721,45.030003,17.769758,14.579918,4.166732,0.225274,-7.021748,13.644271,8.934528,1.298479,5.954477,8.543867,1.117664,0.591559,0.057474,-3.712341,-1.004448,-1.802434,blues
1107,0.248627,0.069145,1188.168337,1682.860150,2339.635853,0.048160,-328.673798,102.696869,19.876062,26.731260,7.158256,22.716642,-6.956797,12.804352,-10.708749,-0.419349,-12.363976,-8.897177,-9.238519,-9.109326,-9.003835,-7.640766,-7.190915,-9.368476,-4.470060,-3.688800,blues
1108,0.269320,0.119072,1361.045467,1567.804596,2739.625101,0.069124,-207.208084,132.799179,-15.438985,60.986729,0.725808,12.437826,1.183836,-1.540780,-17.888153,8.358495,-2.452067,-0.613248,0.384878,2.605127,-5.188924,-9.527454,-9.244394,-2.848273,-1.418707,-5.932607,blues


In [ ]:
genre_list = data1.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)
print(y)
len(y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 

995

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data1.iloc[:, :-1], dtype = float))
X.shape
print(scaler)
import pickle
filename = 'scaler.pkl'
pickle.dump(scaler, open(filename, 'wb'))

StandardScaler()


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = models.Sequential()
X_train[100]

a=[-0.14492835,  1.00400067 , 0.43310275 , 0.54454623,  0.75775479,  0.40733995,
  0.87956002 ,-0.36890081 , 0.34177886  ,0.29173232 ,-0.36187661  ,1.10997991,
 -1.89878167 , 1.11036123 ,-1.38510493  ,0.57681272 ,-1.13252072  ,1.09590156,
  0.13179925 , 1.05871994 ,-0.12250645  ,0.86477539 ,-0.59280286  ,1.38374326,
 -0.36733292 ,-0.23117629]
a = np.array([song])
for i in range(0,10):
   if(X_test[i].all==a.all):
     print(i)
X_test[1]     
y_test[1]

1

In [ ]:
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
              
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=64)

In [ ]:
test_loss, test_acc = model.evaluate(X_test,y_test)
print('test_acc: ',test_loss)
print('test_acc: ',math.ceil(test_acc*100),'%')


7/7 [==============================] - 0s 2ms/step - loss: 2.4692 - accuracy: 0.6985
test_acc:  2.469182252883911
test_acc:  70 %


In [ ]:
a=np.array([[0.260574 	,0.051349 	,1132.340629 	,1582.492047 	,2065.479177 	,0.048314 	,-314.582794 	,139.926743 	,11.865850 	,30.113317 	,-7.549764 	,14.914179 ,	-2.699563 ,	6.707931 ,	-6.391602 	,2.819350 	,-11.715414 ,	2.077480 	,-11.442501 	,-9.504389 	,-15.185495 ,	-9.355572 ,	-5.672771, 	1.564773 	,0.890936 ,-7.960377]])
prediction=loaded_model.predict(a)
print(prediction)

predictio=model.predict(np.array([X_test[0]]))
for i in predictio[0]:
  print(i)
#np.argmax(prediction)#this is being predicted for a single song,thats what we will do when upload...

[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]
-0.30965602
0.33047175
-0.8188771
-0.5376619
-0.6725913
-0.90836275
0.22112507
0.7643878
0.0239295
0.91490227
0.091086015
0.48736507
-0.67819285
0.5858694
-0.5477568
0.51610994
-0.3021774
0.6903372
-0.6240624
0.7500825
-0.37959778
0.60780704
-0.33304045
1.1517762
-0.19088031
-0.58120817


In [ ]:
from keras.models import load_model
model.save("network.h5")


In [ ]:
!gsutil cp   /content/network.h5 /content/drive/MyDrive


Copying file:///content/network.h5...
/ [1 files][609.9 KiB/609.9 KiB]                                                
Operation completed over 1 objects/609.9 KiB.                                    


In [ ]:
loaded_model = load_model("/content/drive/MyDrive/network.h5")
loss, accuracy = loaded_model.evaluate(X_test, y_test)


7/7 [==============================] - 0s 2ms/step - loss: 0.7906 - accuracy: 0.9196


In [ ]:
# predictions = loaded_model.predict(X_test)
# for i in range(0,199):
#   print(np.argmax(predictions[i]))
print(X_test[1])
print(datax)
predictions = loaded_model.predict(X_train)

for i in range(0,100):
  
  print(np.argmax(predictions[i]),y_train[i])  

array([[ 3.87949228e-01,  5.55046536e-02,  2.41193872e+03, ...,
         3.11086130e+00, -4.41013765e+00, -5.03804255e+00],
       [ 2.65696526e-01,  2.99985260e-02,  1.13795424e+03, ...,
         4.25992727e+00,  5.10045767e+00,  2.60898232e+00],
       [ 4.25862879e-01,  1.39735058e-01,  1.80346809e+03, ...,
         5.29056597e+00, -6.59464169e+00, -2.93496609e+00],
       ...,
       [ 5.33298314e-01,  1.19099431e-01,  2.53836806e+03, ...,
         1.62073886e+00, -5.97824335e+00, -8.26951921e-01],
       [ 4.02190477e-01,  1.60759345e-01,  2.80732092e+03, ...,
        -2.50022721e+00, -3.97755718e+00, -8.24019670e-01],
       [ 4.54697639e-01,  2.23418862e-01,  3.12417635e+03, ...,
        -3.13065015e-02, -7.29630709e-01, -9.87610698e-01]])